In [2]:
import pandas as pd

# Paths to the input files
file1_path = "diel/Ag_evap"  # Replace with actual file path
file2_path = "diel/Au_evap"  # Replace with actual file path

# Load data from both files
col_names = ["wave(um)", "Re(n)", "Im(n)", "eps1", "eps2"]
file1_data = pd.read_csv(file1_path, skiprows=3, delim_whitespace=True, names=col_names)
file2_data = pd.read_csv(file2_path, skiprows=3, delim_whitespace=True, names=col_names)

# Align data based on "wave(um)" with a small tolerance
tolerance = 1e-4  # Small tolerance for matching
aligned_data = pd.merge_asof(
    file1_data.sort_values("wave(um)"),
    file2_data.sort_values("wave(um)"),
    on="wave(um)",
    direction="nearest",
    suffixes=("_Ag", "_Au"),
    tolerance=tolerance
).dropna()  # Drop rows where matching failed

# Function to create a combined dielectric file
def create_combined_dielectric_file(df, ratio_Ag, ratio_Au, filename):
    combined = pd.DataFrame({
        "wave(um)": df["wave(um)"],
        "Re(n)": ratio_Ag * df["Re(n)_Ag"] + ratio_Au * df["Re(n)_Au"],
        "Im(n)": ratio_Ag * df["Im(n)_Ag"] + ratio_Au * df["Im(n)_Au"],
        "eps1": ratio_Ag * df["eps1_Ag"] + ratio_Au * df["eps1_Au"],
        "eps2": ratio_Ag * df["eps2_Ag"] + ratio_Au * df["eps2_Au"],
    })
    
    # Write the file in the required format
    with open(filename, 'w') as f:
        f.write(f"Combined dielectric data ({int(ratio_Au*100)}Au/{int(ratio_Ag*100)}Ag), Alexander Al-Feghali al-fegha.li\n")
        f.write("1 2 3 0 0 = columns for wave, Re(n), Im(n), eps1, eps2\n")
        f.write("wave(um) Re(n)  Im(n)    eps1   eps2\n")
        combined.to_csv(f, sep='\t', index=False, header=False, float_format="%.6f")

# File outputs
create_combined_dielectric_file(aligned_data, 0.75, 0.25, "25Au75Ag_dielec")
create_combined_dielectric_file(aligned_data, 0.50, 0.50, "50Au50Ag_dielec")
create_combined_dielectric_file(aligned_data, 0.25, 0.75, "75Au25Ag_dielec")

print("Files generated: 25Au75Ag, 50Au50Ag, 75Au25Ag")


Files generated: 25Au75Ag, 50Au50Ag, 75Au25Ag


In [ ]:
create_combined_dielectric_file(aligned_data, 1, 0, "Ag_test")

: 